In [6]:
!git clone https://github.com/Kahkashan2708/RAG-voice-chatbot.git
%cd RAG-voice-chatbot

Cloning into 'RAG-voice-chatbot'...
remote: Enumerating objects: 2963, done.
remote: Counting objects: 100% (2963/2963), done.
remote: Compressing objects: 100% (2239/2239), done.
^C
[Errno 2] No such file or directory: 'RAG-voice-chatbot'
/content/NeMo


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/MyDrive/indicconformer_stt_hi_hybrid_rnnt_large.nemo" \
    "/content/RAG-voice-chatbot/models/"

## Install Libraries

In [1]:
! pip install --upgrade pip

In [2]:
! pip install numpy==1.26.4 scipy==1.11.4

In [17]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 160.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 42.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 176.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 16.6 MB/s  0:00:19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 56.7 MB/s  0:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 69.9 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 51.1 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 59.8 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 72.9 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 69.7 MB/s  0:00:02
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a whil

In [4]:
! pip install nemo_toolkit[asr]

  Using cached nemo_toolkit-2.6.1-py3-none-any.whl.metadata (81 kB)
Using cached nemo_toolkit-2.6.1-py3-none-any.whl (5.6 MB)


In [5]:
! pip install soundfile pydub librosa

In [6]:
! pip install fastapi uvicorn python-multipart

In [7]:
import torch
import nemo.collections.asr as nemo_asr
import fastapi

print("Torch:", torch.__version__)
print("NeMo ASR OK")
print("FastAPI OK")

[NeMo W 2026-01-24 15:10:42 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.


Torch: 2.9.0+cpu
NeMo ASR OK
FastAPI OK


In [9]:
!git clone https://github.com/AI4Bharat/NeMo.git
%cd NeMo
!git checkout nemo-v2
!bash reinstall.sh

fatal: destination path 'NeMo' already exists and is not an empty directory.
/content/NeMo
Already on 'nemo-v2'
Your branch is up to date with 'origin/nemo-v2'.
Uninstalling stuff
Found existing installation: nemo-toolkit 2.6.1
Uninstalling nemo-toolkit-2.6.1:
  Successfully uninstalled nemo-toolkit-2.6.1
Installing nemo
Obtaining file:///content/NeMo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
  Using cached triton-3.6.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
  Using cached black-19.10b0-py36-none-any.whl.metadata (58 kB)
  Using cached click-8.0.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached isort-5.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached parameterized-0.9.0-py2.py3-none-any.whl.metadata (18 kB)

In [2]:
import sys
sys.path.insert(0, "/content/NeMo")

In [3]:
import nemo
print(nemo.__file__)

/content/NeMo/nemo/__init__.py


In [4]:
import torch
import nemo.collections.asr as nemo_asr

model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.restore_from(
    "/content/RAG-voice-chatbot/models/indicconformer_stt_hi_hybrid_rnnt_large.nemo",
    map_location="cuda" if torch.cuda.is_available() else "cpu"
)

model.eval()
print("✅ IndicConformer model loaded successfully")

[NeMo W 2026-01-24 16:52:48 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/megatron/core/tensor_parallel/layers.py:219: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2026-01-24 16:52:48 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/megatron/core/tensor_parallel/layers.py:249: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      @custom_bwd
    
[NeMo W 2026-01-24 16:52:58 nemo_logging:393] /content/NeMo/nemo/collections/asr/modules/audio_modules.py:1440: SyntaxWarning: invalid escape sequence '\ '
      G(b,f) = Q(b,f) \ R(b,f)
    
[NeMo W 2026-01-24 16:52:59 nemo_logging:393] /content/NeMo/nemo/collections/asr/parts/utils/vad_utils.py:1093: SyntaxWarning: invalid escape sequence '\s'
      data = pd.read_csv(path2ground_truth_label, sep="\s+", delim

[NeMo I 2026-01-24 16:53:12 nemo_logging:381] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:53:12 nemo_logging:381] To

[NeMo W 2026-01-24 16:53:19 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - /nlsasfs/home/ai4bharat/ai4bharat-pr/speechteam/indicasr_v3/manifests/nemo/vistaar_v3/train/train_hindi.json
    sample_rate: 16000
    batch_size: 8
    num_workers: 16
    pin_memory: true
    max_duration: 30.0
    min_duration: 0.2
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    is_concat: true
    concat_sampling_technique: temperature
    concat_sampling_temperature: 1.5
    return_language_id: true
    
[NeMo W 2026-01-24 16:53:19 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to 

[NeMo I 2026-01-24 16:53:19 nemo_logging:381] PADDING: 0


[NeMo W 2026-01-24 16:53:21 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn(
    


[NeMo I 2026-01-24 16:53:23 nemo_logging:381] Vocab size for each language: 256
[NeMo I 2026-01-24 16:53:23 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2026-01-24 16:53:23 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2026-01-24 16:53:25 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2026-01-24 16:53:25 nemo_logging:381] Creating masks for multi-softmax layer.
[NeMo I 2026-01-24 16:53:25 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2026-01-24 16:53:28 nemo_logging:381] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /content/RAG-voice-chatbot/models/indicconformer_stt_hi_hybrid_rnnt_large.nemo.
✅ IndicConformer model loaded successfu

In [5]:
def transcribe_audio(audio_path: str):
    with torch.no_grad():
        text = model.transcribe(
            audio_path,
            batch_size=1,
            language_id="hi"   # Hindi
        )
    return text[0]

In [6]:
text = transcribe_audio("/content/RAG-voice-chatbot/sample.wav")
print("Transcription:", text)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s][NeMo W 2026-01-24 16:53:39 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
      warnings.warn(warn_msg)
    
[NeMo W 2026-01-24 16:53:57 nemo_logging:393] /content/NeMo/nemo/collections/asr/parts/preprocessing/features.py:417: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      with torch.cuda.amp.autocast(enabled=False):
    
Transcribing: 100%|██████████| 1/1 [00:20<00:00, 20.40s/it]

Transcription: [' नमस्कार मुझे आर्टिफिशियल इंटेलिजेंस के बारे में बताएं धन्यवाद']


In [7]:
%cd /content/RAG-voice-chatbot

/content/RAG-voice-chatbot


In [ ]:
import sys
sys.path.insert(0, "/content/NeMo")

In [8]:
%%writefile scripts/task3_asr.py
import sys
sys.path.insert(0, "/content/NeMo")
from fastapi import FastAPI, UploadFile, File
import tempfile
import os
import torch
import nemo.collections.asr as nemo_asr


# FastAPI app

app = FastAPI(
    title="Hindi ASR Service",
    description="ASR using AI4Bharat IndicConformer (NeMo)",
    version="1.0"
)


# Load IndicConformer model
MODEL_PATH = "/content/RAG-voice-chatbot/models/indicconformer_stt_hi_hybrid_rnnt_large.nemo"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.restore_from(
    restore_path=MODEL_PATH,
    map_location=DEVICE
)
model.eval()


# Health check
@app.get("/")
def health():
    return {"status": "IndicConformer ASR service running"}


# Transcription endpoint
@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    # Save uploaded audio temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        tmp.write(await file.read())
        audio_path = tmp.name

    try:
        with torch.no_grad():
            transcription = model.transcribe(
                audio=[audio_path],
                language_id="hi"   # Hindi
            )

        return {
            "language": "hi",
            "transcription": transcription[0]
        }

    finally:
        os.remove(audio_path)

Overwriting scripts/task3_asr.py


In [9]:
!uvicorn scripts.task3_asr:app --host 0.0.0.0 --port 8000

[NeMo W 2026-01-24 16:59:11 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/megatron/core/tensor_parallel/layers.py:219: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2026-01-24 16:59:11 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/megatron/core/tensor_parallel/layers.py:249: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      @custom_bwd
    
[NeMo I 2026-01-24 16:59:29 nemo_logging:381] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2026-01-24 16:59:29 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:59:29 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 16:59:29 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 toke

In [13]:
import requests
requests.get("http://127.0.0.1:8000/").json()

{'status': 'IndicConformer ASR service running'}

In [14]:
import requests

files = {
    "file": open("/content/RAG-voice-chatbot/sample.wav", "rb")
}

res = requests.post("http://127.0.0.1:8000/transcribe", files=files)
print(res.json())

{'language': 'hi', 'transcription': [' नमस्कार मुझे आर्टिफिशियल इंटेलिजेंस के बारे में बताएं धन्यवाद']}


## Task-5

In [16]:
%cd /content/RAG-voice-chatbot

/content/RAG-voice-chatbot


In [18]:
import sys
sys.path.append("/content/RAG-voice-chatbot/scripts")

In [15]:
! pip install sarvamai

In [20]:
from task4_translate import translate_to_english

print(translate_to_english("मुझे आर्टिफिशियल इंटेलिजेंस के बारे में बताइए"))


[NeMo W 2026-01-24 17:17:41 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
    The secret `HF_TOKEN` does not exist in your Colab secrets.
    To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
    You will be able to reuse this secret in all of your notebooks.
    Please note that authentication is recommended but still optional to access public models or datasets.
      warnings.warn(
    


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Tell me about Artificial Integration


In [21]:
! pip install -U \ langchain-community \ langchain \ sentence-transformers \ faiss-cpu

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s  0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: marshmallow
    Found existing installation: marshmallow 4.2.1
    Uninstalling marshmallow-4.2.1:
      Successfully uninstalled marshmallow-4.2.1
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.6
    Uninstalling langgraph-prebuilt-1.0.6:
      Successfully uninstalled langgraph-prebuilt-1.0.6
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.6
    Uninstalling langgraph-1.0.6:
      Successfully uninstalled langgraph-1.0.6
  Attempting uninstall: langchain

In [2]:
!pip uninstall -y transformers huggingface_hub sentence-transformers langchain langchain-community langchain-huggingface
!pip cache purge

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: huggingface-hub 0.23.2
Uninstalling huggingface-hub-0.23.2:
  Successfully uninstalled huggingface-hub-0.23.2
Found existing installation: sentence-transformers 5.2.0
Uninstalling sentence-transformers-5.2.0:
  Successfully uninstalled sentence-transformers-5.2.0
Found existing installation: langchain 1.2.7
Uninstalling langchain-1.2.7:
  Successfully uninstalled langchain-1.2.7
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Files removed: 1332 (3579.0 MB)


In [3]:
!pip install --no-cache-dir \
huggingface_hub==0.22.2 \
transformers==4.38.2 \
sentence-transformers==2.6.1 \
langchain==0.1.20 \
langchain-community==0.0.38

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 31.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.2 MB/s  0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.6.4
    Uninstalling langsmith-0.6.4:
      Successfully uninstalled langsmith-0.6.4
  Attempting uninstall: langchain-core
    Found

In [1]:
import transformers, huggingface_hub, langchain
print(transformers.__version__)
print(huggingface_hub.__version__)
print(langchain.__version__)

4.38.2
0.22.2
0.1.20


In [54]:
%%writefile scripts/task5_rag_pipeline.py
import os
from typing import List

import torch
import nemo.collections.asr as nemo_asr

from scripts.task4_translate import translate_to_english
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import requests


# --------------------------------------------------
# CONFIG
# --------------------------------------------------

VECTOR_DB_PATH = "vector_db/faiss_index"

# MUST be same as Task-2 embedding model
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

ASR_MODEL_PATH = "models/indicconformer_stt_hi_hybrid_rnnt_large.nemo"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# --------------------------------------------------
# LOAD ASR MODEL (DIRECT, NO FASTAPI)
# --------------------------------------------------

asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.restore_from(
    restore_path=ASR_MODEL_PATH,
    map_location=DEVICE
)
asr_model.eval()


def transcribe_audio(audio_file_path: str) -> str:
    """
    Transcribe Hindi audio using IndicConformer (NeMo)
    """

    if not os.path.exists(audio_file_path):
        raise FileNotFoundError(f"Audio file not found: {audio_file_path}")

    with torch.no_grad():
        text = asr_model.transcribe(
            audio=[audio_file_path],
            language_id="hi"
        )

    return text[0]


# --------------------------------------------------
# VECTOR DB RETRIEVAL
# --------------------------------------------------

def retrieve_relevant_chunks(
    query: str,
    top_k: int = 2
) -> List[str]:
    """
    Retrieve top-k relevant chunks from FAISS vector DB
    """

    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME
    )

    vector_store = FAISS.load_local(
        VECTOR_DB_PATH,
        embeddings,
        allow_dangerous_deserialization=True
    )

    docs = vector_store.similarity_search(query, k=top_k)

    return [doc.page_content for doc in docs]


# --------------------------------------------------
# LLM: GitHub Models
# --------------------------------------------------

def generate_answer_from_llm(
    question: str,
    context_chunks: List[str]
) -> str:
    """
    Generate answer using GitHub Models (OpenAI compatible API)
    """

    github_token = os.getenv("GITHUB_TOKEN")
    if not github_token:
        raise EnvironmentError("GITHUB_TOKEN not found in environment")

    api_url = "https://models.inference.ai.azure.com/chat/completions"

    headers = {
        "Authorization": f"Bearer {github_token}",
        "Content-Type": "application/json"
    }

    context = "\n\n".join(context_chunks)

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "system",
                "content": "Answer strictly using the provided context."
            },
            {
                "role": "user",
                "content": f"Context:\n{context}\n\nQuestion:\n{question}"
            }
        ],
        "temperature": 0.3
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code != 200:
        raise RuntimeError(
            f"LLM API failed {response.status_code}: {response.text}"
        )

    return response.json()["choices"][0]["message"]["content"]


# --------------------------------------------------
# MAIN RAG PIPELINE
# --------------------------------------------------

def run_rag_pipeline(audio_file_path: str) -> str:
    """
    Audio → ASR → Translation → Retrieval → LLM → Answer
    """

    # Step 1: ASR
    hindi_text = transcribe_audio(audio_file_path)
    print("\nASR Output:\n", hindi_text)

    # Step 2: Translate
    english_question = translate_to_english(hindi_text)
    print("\nTranslated Question:\n", english_question)

    # Step 3: Retrieve context
    context_chunks = retrieve_relevant_chunks(english_question)

    # Step 4: Generate answer
    final_answer = generate_answer_from_llm(
        english_question,
        context_chunks
    )

    return final_answer


# --------------------------------------------------
# LOCAL TEST
# --------------------------------------------------

if __name__ == "__main__":
    audio_path = "sample.wav"
    answer = run_rag_pipeline(audio_path)

    print("\nFinal Answer:\n")
    print(answer)


Overwriting scripts/task5_rag_pipeline.py


In [12]:
import nemo
print(nemo.__file__)

/content/NeMo/nemo/__init__.py


In [13]:
import sys
sys.path.insert(0, "/content/NeMo")

from scripts.task5_rag_pipeline import run_rag_pipeline
answer = run_rag_pipeline("sample.wav")
print("\nFINAL ANSWER:\n", answer)

[NeMo W 2026-01-24 17:28:40 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/megatron/core/tensor_parallel/layers.py:219: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2026-01-24 17:28:40 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/megatron/core/tensor_parallel/layers.py:249: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      @custom_bwd
    
[NeMo W 2026-01-24 17:29:04 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
    The secret `HF_TOKEN` does not exist in your Colab secrets.
    To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
    You will be able to r

[NeMo I 2026-01-24 17:29:19 nemo_logging:381] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2026-01-24 17:29:19 nemo_logging:381] To

[NeMo W 2026-01-24 17:29:27 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - /nlsasfs/home/ai4bharat/ai4bharat-pr/speechteam/indicasr_v3/manifests/nemo/vistaar_v3/train/train_hindi.json
    sample_rate: 16000
    batch_size: 8
    num_workers: 16
    pin_memory: true
    max_duration: 30.0
    min_duration: 0.2
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    is_concat: true
    concat_sampling_technique: temperature
    concat_sampling_temperature: 1.5
    return_language_id: true
    
[NeMo W 2026-01-24 17:29:27 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to 

[NeMo I 2026-01-24 17:29:27 nemo_logging:381] PADDING: 0


[NeMo W 2026-01-24 17:29:29 nemo_logging:393] /usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn(
    


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


FINAL ANSWER:
 The provided context does not mention "Artificial Investigation." It focuses on artificial general intelligence and the evolving perspectives on risks associated with superintelligent machines. If you have specific questions about artificial general intelligence or related topics, feel free to ask!


## app.py

In [53]:
%%writefile scripts/app.py
import os
import sys
os.environ["GITHUB_TOKEN"] = "XXXXXXXXXXXXXXXXXXXXXXX"
sys.path.insert(0, "/content/NeMo")
import gradio as gr
from task5_rag_pipeline import run_rag_pipeline


def voice_chatbot(audio_file):
    """
    Takes audio file path from Gradio,
    runs full RAG pipeline,
    returns final answer text.
    """
    if audio_file is None:
        return "❌ Please upload an audio file (.wav)."

    try:
        answer = run_rag_pipeline(audio_file)
        return answer
    except Exception as e:
        return f"⚠️ Error occurred:\n{str(e)}"


with gr.Blocks(
    title="Voice-enabled RAG Chatbot",
    theme=gr.themes.Soft()
) as demo:

    gr.Markdown(
        """
        # 🎙️ Voice-enabled RAG Chatbot
        ### Hindi Speech → AI-powered Answer

        This application performs:
        - 🎧 **Automatic Speech Recognition (IndicConformer)**
        - 🌐 **Hindi → English Translation**
        - 📚 **Context Retrieval using FAISS Vector DB**
        - 🤖 **Answer Generation using LLM (GitHub Models)**

        Upload a **Hindi audio (.wav)** file and get an intelligent answer.
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            audio_input = gr.Audio(
                type="filepath",
                label="🎧 Upload Audio (.wav)",
                interactive=True
            )

            submit_btn = gr.Button(
                "🚀 Ask Question",
                variant="primary"
            )

        with gr.Column(scale=1):
            output_text = gr.Textbox(
                label="🤖 Chatbot Answer",
                lines=10,
                interactive=False,
                placeholder="The answer will appear here..."
            )

    submit_btn.click(
        fn=voice_chatbot,
        inputs=audio_input,
        outputs=output_text
    )

    gr.Markdown(
        """
        ---
        **Tech Stack:**
        NeMo IndicConformer · FastAPI · FAISS · LangChain · Gradio · GitHub Models
        """
    )
demo.launch(share=True)

Overwriting scripts/app.py


In [55]:
%cd scripts

/content/RAG-voice-chatbot/scripts
